# Нейронная сеть с нуля.

Начнем с нужных библиотек: `numpy`

In [1]:
import numpy as np

## 1. Класс сети.

Мне хотелось реализовать отдельный класс для нейронной сети чтобы выглядело хоть сколько-нибудь серьезно. Однако учитывая, что в этой сети всегда будет два нейрона (входной не считаем) особой серьезностью тут не пахнет, откуда и название:

In [3]:
class BasicNet:

    def __init__(self, input_w, input_b, output_w, output_b, inputs, outputs):
        self.hidden_w = input_w
        self.hidden_b = input_b
        self.out_w = output_w
        self.out_b = output_b
        self.X = inputs
        self.y = outputs
        self.results = [np.empty(2), np.empty(1)]

    def forward(self):
        self.results[0] = sigmoid(np.dot(self.X, self.hidden_w) + np.array([self.hidden_b] * 4).reshape(4, 2))
        self.results[1] = sigmoid(np.dot(self.results[0], self.out_w) + np.array([self.out_b] * 4).reshape(4, 1))
        return self.results[1]

    def backward(self):
        d_outw = np.dot(self.results[0].T, (self.y - self.results[1]) * sigmoid_derivative(self.results[1]))
        d_outb = np.sum((self.y - self.results[1]) * sigmoid_derivative(self.results[1]))

        d_inter = np.dot((self.y - self.results[1]) * sigmoid_derivative(self.results[1]), self.out_w.T)

        d_hiddenw = np.dot(self.X.T, d_inter * sigmoid_derivative(self.results[0]))
        d_hiddenb = np.sum(d_inter * sigmoid_derivative(self.results[0]))

        self.out_w += d_outw
        self.out_b += d_outb

        self.hidden_w += d_hiddenw
        self.hidden_b += d_hiddenb


In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    return x * (1 - x)


Сигмоиду с ее производной пришлось занести в отдельный файл. Объясним как все работает.

## 2. Инициализация.

Мы не будем делать хоть сколько-нибудь гибкую сеть для избежания ненужного усложнения кода, так что в инициализацию запихаем все веса, байесы, и даже вход с нужным выходом для обучения. Так же заведем массив `results` промежуточных выходов, слоя два так что и выхода будет 2.

## 3. Forward.

*forward propagation* реализовать как раз не очень сложно. Достаточно для каждого нейрона с весами `w`, байесом `b` сделать следующую функцию активации

$$ f(x)=\sigma(w\cdot x + b). $$

Далее просто протаскиваем `input` через эти функции по очереди и получаем что хотели, запоминая промежуточные выходы, это секретный инструмент который пригодится далее.

## 4. Backward.

Теперь немного сложнее. Для *backward propagation* мы хотим двигаться в сторону локального минимума, так что каждый вес и байес было бы классно изменить против градиента.

Распишем для примера вычисления для производной по весам:

$$
    \frac{\partial}{\partial w} \operatorname{Loss} = \frac{\partial L}{\partial y_{pred}} \cdot \frac{\partial y_{pred}}{\partial (wx+b)} \cdot \frac{\partial(wx+b)}{\partial w}=
$$

$$
    =2(y-y_pred) \cdot \operatorname{sigmoid_derivative}(wx+b) \cdot x.
$$

Для `bias` все будет так же, разве что нет x и придется брать сумму, так как выходит вектор. Для производных весов и байесов нужно будет протянуть промежуточную производную, если бы она была именно такой, то достаточно было просто домножить, но у меня долго не сходились размерности матриц, так что пришлось писать руками. Это, к слову, причина почему я не реализовывал отдельный класс `neuron`. Из тонкостей остался тот факт, что нам дается не один `x`, а список `x`'ов, так что придется пару тройку раз делать транспозиции и менять местами матрицы в произведении.

## 5. Тестирование.
Теперь код очень прост, достаточно просто создать сеть, вставить нужные применения соответствующих методов и готово!

In [5]:
import sys
import datetime

def print_with_datetime(s):
    time_string = datetime.datetime.now().strftime("%H:%M:%S")
    sys.stdout.write("\r" + time_string + " " + s)
    sys.stdout.flush()


# Input datasets
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
target = np.array([[0], [1], [1], [0]])

epochs = 10000
lr = 0.1
inputLayerNeurons, hiddenLayerNeurons, outputLayerNeurons = 2, 2, 1

# Random weights and bias initialization
hidden_weights = np.random.uniform(size=(inputLayerNeurons, hiddenLayerNeurons))
hidden_bias = np.random.uniform(size=(1, hiddenLayerNeurons))
output_weights = np.random.uniform(size=(hiddenLayerNeurons, outputLayerNeurons))
output_bias = np.random.uniform(size=(1, outputLayerNeurons))

print("Initial hidden weights: ", end='')
print(*hidden_weights)
print("Initial hidden biases: ", end='')
print(*hidden_bias)
print("Initial output weights: ", end='')
print(*output_weights)
print("Initial output biases: ", end='')
print(*output_bias)



net = BasicNet(hidden_weights, hidden_bias, output_weights, output_bias, inputs, target)

# Training algorithm
for epoch in range(epochs):
    # Forward Propagation
    # hidden_outputs = ...
    predicted_output = net.forward()

    # Loss
    loss = 0.5 * (target - predicted_output) ** 2
    loss = loss.sum()
    #print_with_datetime("Epoch {} Loss {:.4f}".format(epoch, loss)) pycharm очень не хочет выводить 10000 строчек, а как поменять хз

    # Backpropagation
    # loss_by_output = ...
    # predicted_output_derivative = ...

    # loss_by_output_bias = ...

    # loss_by_output_weights = ...

    # loss_by_hidden_outputs = ...

    # hidden_outputs_derivative = ...

    # loss_by_hidden_weights = ...

    # Updating Weights and Biases
    # output_bias -= ...
    # output_weights -= ...
    # hidden_bias -= ...
    # hidden_weights -= ...
    net.backward()

print('')
print("Final hidden weights: ", end='')
print(*hidden_weights)
print("Final hidden bias: ", end='')
print(*hidden_bias)
print("Final output weights: ", end='')
print(*output_weights)
print("Final output bias: ", end='')
print(*output_bias)

print("\nOutput from neural network after 10,000 epochs: ", end='')
print(*predicted_output)

Initial hidden weights: [0.07919192 0.49241169] [0.74524372 0.91863595]
Initial hidden biases: [0.46682091 0.39463166]
Initial output weights: [0.98496711] [0.19460319]
Initial output biases: [0.47726976]

Final hidden weights: [9.14011262 3.99417503] [9.14203286 3.99467789]
Final hidden bias: [-5.92031805 -5.9925073 ]
Final output weights: [10.58505394] [-12.07660121]
Final output bias: [-4.30939596]

Output from neural network after 10,000 epochs: [0.01324127] [0.98819402] [0.98819271] [0.01265804]


## 6. Заключение.

Несмотря на то, что реализовать настолько простую нейросеть не составляет труда, некоторые проблемы ускользают от моего понимания, например как реализовывать нейроны и тд.